In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

--2021-01-27 12:06:12--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84199 (82K) [application/x-httpd-php]
Saving to: ‘winequality-red.csv’

winequality-red.csv 100%[===================>]  82.23K   273KB/s    in 0.3s    

2021-01-27 12:06:13 (273 KB/s) - ‘winequality-red.csv’ saved [84199/84199]



In [59]:
import pandas as pd

In [60]:
data=pd.read_csv('/content/winequality-red.csv',sep=';')
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [61]:
features = data.columns
features=list(features)

In [62]:
feat_data=data[features]

In [63]:
from xgboost import XGBClassifier
from sklearn. model_selection import train_test_split

from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier,RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score

In [64]:
labels=feat_data.pop('quality')
X_train,X_test,y_train,y_test=train_test_split(data,labels,test_size=0.2,random_state=42)

In [69]:
KNN=KNeighborsClassifier()
KNN.fit(X_train,y_train)
pred=KNN.predict(X_test)

print("KNeighborsClassifier accuracy is :{}".format(accuracy_score(y_test,pred)))

KNeighborsClassifier accuracy is :0.58125


In [85]:
Bagging=BaggingClassifier(n_estimators=100)
Bagging.fit(X_train,y_train)
pred=Bagging.predict(X_test)

print("BaggingClassifier accuracy is :{}".format(accuracy_score(y_test,pred)))

BaggingClassifier accuracy is :1.0


In [70]:
svm=SVC(gamma='scale', probability=True)
svm.fit(X_train,y_train)
svm_pred=svm.predict(X_test)

print("SVM accuracy is :{}".format(accuracy_score(y_test,svm_pred)))

SVM accuracy is :0.5375


In [86]:
random_f=RandomForestClassifier(n_estimators=150)
random_f.fit(X_train,y_train)
random_f_pred=random_f.predict(X_test)

print("random forest accuracy is :{}".format(accuracy_score(y_test,random_f_pred)))

random forest accuracy is :0.98125


In [74]:
log=LogisticRegression(solver='liblinear')
log.fit(X_train,y_train)
pred=log.predict(X_test)

print("LogisticRegression accuracy is :{}".format(accuracy_score(y_test,pred)))

LogisticRegression accuracy is :0.896875


In [76]:
Decision=DecisionTreeClassifier()
Decision.fit(X_train,y_train)
pred=Decision.predict(X_test)

print("DecisionTreeClassifier accuracy is :{}".format(accuracy_score(y_test,pred)))

DecisionTreeClassifier accuracy is :1.0


In [77]:
guassian=GaussianNB()
guassian.fit(X_train,y_train)
pred=guassian.predict(X_test)

print("GaussianNB accuracy is :{}".format(accuracy_score(y_test,pred)))

GaussianNB accuracy is :1.0


In [78]:
Ada=AdaBoostClassifier()
Ada.fit(X_train,y_train)
pred=Ada.predict(X_test)

print("AdaBoostClassifier accuracy is :{}".format(accuracy_score(y_test,pred)))

AdaBoostClassifier accuracy is :0.8375


In [81]:
Ex_Tree=ExtraTreesClassifier(n_estimators=100)
Ex_Tree.fit(X_train,y_train)
pred=Ex_Tree.predict(X_test)

print("ExtraTreesClassifier accuracy is :{}".format(accuracy_score(y_test,pred)))

ExtraTreesClassifier accuracy is :0.996875


In [82]:
XGB=XGBClassifier()
XGB.fit(X_train,y_train)
pred=XGB.predict(X_test)

print("XGBClassifier accuracy is :{}".format(accuracy_score(y_test,pred)))

XGBClassifier accuracy is :1.0


In [91]:
def get_models():
    models = list()
    models.append(KNeighborsClassifier())
    models.append(BaggingClassifier(n_estimators=10))
    models.append(SVC(gamma='scale', probability=True))
    models.append(RandomForestClassifier(n_estimators=10))
    models.append(LogisticRegression(solver='liblinear'))
    models.append(DecisionTreeClassifier())
    models.append(GaussianNB())
    models.append(AdaBoostClassifier())
    models.append(ExtraTreesClassifier(n_estimators=10))
    models.append(XGBClassifier())
    return models

In [92]:
!pip install mlens

In [99]:
import mlens
from mlens.ensemble import SuperLearner
def get_super_learner(X):
	ensemble = SuperLearner(scorer=accuracy_score, folds=10, shuffle=True, sample_size=len(X))
	models = get_models()
	ensemble.add(models)
	ensemble.add_meta(LogisticRegression(solver='liblinear'))
	return ensemble

In [100]:
ensemble = get_super_learner(X_train)

In [101]:
ensemble.fit(X_train.values,y_train.values)
print(ensemble.data)

yhat = ensemble.predict(X_test.values)

print("*"* 50)
score=accuracy_score(y_test,yhat)
print("Super Learner accuracy is :{}".format(score))

/usr/local/lib/python3.6/dist-packages/mlens/parallel/_base_functions.py:313: ParameterChangeWarning: Parameter value ('multi:softprob') has changed since model was fitted ('binary:logistic').
  (lpar, rpar), ParameterChangeWarning)
/usr/local/lib/python3.6/dist-packages/mlens/parallel/_base_functions.py:313: ParameterChangeWarning: Parameter value ('multi:softprob') has changed since model was fitted ('binary:logistic').
  (lpar, rpar), ParameterChangeWarning)


                                   score-m  score-s  ft-m  ft-s  pt-m  pt-s
layer-1  adaboostclassifier           0.84     0.03  0.29  0.01  0.03  0.00
layer-1  baggingclassifier            1.00     0.00  0.05  0.00  0.00  0.00
layer-1  decisiontreeclassifier       1.00     0.00  0.00  0.00  0.00  0.00
layer-1  extratreesclassifier         0.99     0.01  0.03  0.01  0.00  0.00
layer-1  gaussiannb                   1.00     0.00  0.00  0.00  0.00  0.00
layer-1  kneighborsclassifier         0.59     0.03  0.00  0.00  0.02  0.00
layer-1  logisticregression           0.89     0.03  0.05  0.00  0.00  0.00
layer-1  randomforestclassifier       0.98     0.01  0.05  0.00  0.00  0.00
layer-1  svc                          0.53     0.05  0.65  0.01  0.01  0.00
layer-1  xgbclassifier                1.00     0.00  0.43  0.01  0.00  0.00

**************************************************
Super Learner accuracy is :0.978125
